<a href="https://colab.research.google.com/github/manan-arya/Major_Project/blob/pso%2B/Bat_Optimaization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [2]:
! pip install pyswarms
import pyswarms as ps

     |████████████████████████████████| 102kB 5.5MB/s 


In [0]:
url = "https://raw.githubusercontent.com/saranshtaneja/genetic_major/master/fb_dataset.csv"
data = pd.read_csv(url)

In [4]:
data

,facebook_post_id,dominant_color.histogram[0],dominant_color.histogram[1],dominant_color.histogram[2],dominant_color.histogram[3],dominant_color.histogram[4],dominant_color.histogram[5],dominant_color.histogram[6],dominant_color.histogram[7],dominant_color.histogram[8],dominant_color.histogram[9],dominant_color.value,face_detection.average_face2frame_ratio,face_detection.average_face_count_per_frame,face_detection.average_face_present,general.duration,general.fps,general.frame_count,general.frame_size.height,general.frame_size.width,other.average_movie_speed,other.clutter_metric,rigidity.average_rigidity,shot_detection.average_shot_length,shot_detection.shots_count,shot_detection.transition_histogram[0],shot_detection.transition_histogram[1],text_detection.average_text2frame_ratio,text_detection.average_text_frames,title.sentiment,views.1,shares.1,comments.1,likes.1,views.2,shares.2,comments.2,likes.2,views.3,shares.3,...,views.159,shares.159,comments.159,likes.159,views.160,shares.160,comments.160,likes.160,views.161,shares.161,comments.161,likes.161,views.162,shares.162,comments.162,likes.162,views.163,shares.163,comments.163,likes.163,views.164,shares.164,comments.164,likes.164,views.165,shares.165,comments.165,likes.165,views.166,shares.166,comments.166,likes.166,views.167,shares.167,comments.167,likes.167,shares.168,comments.168,likes.168,views.168
0,407570359384477_612049142269930,0.002330,0.285182,0.321528,0.191053,0.000000,0.000000,0.000000,0.007922,0.093197,0.098788,2,0.015108,0.251165,0.243243,85.840000,25,2146,224,400,1.680012,0.161410,0.212954,4.517895,19,1.000000,0.000000,0.054041,0.446412,4.651,21057,370,69,652,36707,619,105,1000,48639,796,...,350437,4913,376,4223,350504,4914,376,4223,350577,4915,376,4223,350642,4915,376,4223,350716,4916,376,4223,350785,4916,376,4223,350858,4916,376,4224,350942,4916,376,4225,351045,4916,376,4225,4916,376,4225,351137
1,407570359384477_612108412264003,0.005907,0.463291,0.059072,0.192405,0.000000,0.000000,0.000000,0.093671,0.157806,0.027848,1,0.009970,0.304641,0.098734,47.400000,25,1185,224,400,15.270662,0.106316,0.270886,3.950000,12,0.833333,0.166667,0.052529,0.451477,4.566,15618,168,27,475,22384,246,33,690,24171,292,...,87968,578,61,1617,87998,578,61,1617,88020,578,61,1617,88029,578,61,1617,88042,578,61,1617,88066,578,61,1617,88094,578,61,1617,88110,579,61,1617,88128,579,61,1617,579,61,1617,88149
2,407570359384477_612000682274776,0.000000,0.377131,0.072443,0.093750,0.000000,0.000000,0.000000,0.197443,0.125000,0.134233,1,0.014187,0.171165,0.171165,56.320000,25,1408,224,400,4.000194,0.188503,0.387074,4.693333,12,0.750000,0.250000,0.084811,0.714489,3.721,2,81,67,329,35031,109,83,437,41947,123,...,116446,329,170,1193,116454,329,170,1193,116462,329,170,1193,116479,329,170,1193,116504,329,170,1193,116521,329,170,1193,116545,329,170,1193,116570,329,170,1193,116582,329,170,1193,329,170,1193,116597
3,407570359384477_612254708916040,0.000000,0.592656,0.144475,0.079272,0.072066,0.012697,0.000000,0.097117,0.001716,0.000000,1,0.006673,0.067605,0.066575,116.560000,25,2914,224,400,5.159395,0.098545,0.355525,5.067826,23,0.739130,0.260870,0.035462,0.315717,3.851,89140,4407,235,3486,213383,9438,402,6044,347713,14571,...,18606897,464169,7616,121156,18613170,464233,7616,121198,18619570,464290,7619,121244,18626146,464343,7622,121288,18632836,464385,7624,121330,18638793,464418,7626,121367,18643654,464447,7627,121397,18647758,464489,7628,121419,18651565,464539,7631,121442,464591,7633,121472,18655415
4,407570359384477_612167392258105,0.000000,0.116180,0.436074,0.154377,0.000000,0.000000,0.000000,0.293369,0.000000,0.000000,2,0.008844,0.094430,0.094430,75.400000,25,1885,224,400,5.295101,0.109216,0.611141,4.712500,16,0.812500,0.187500,0.049729,0.327321,4.901,17304,240,127,233,26215,355,176,337,29539,438,...,432502,6536,1032,1513,432868,6541,1032,1514,433204,6543,1032,1515,433517,6548,1032,1515,433862,6556,1032,1516,434279,6565,1033,1517,434799,6573,1033,1517,435447,6581,1035,1517,436017,6589,1035,1517,6597,

In [0]:
#  DATA SELECTOR

x_train = data.iloc[0:100,1:701].values
x_test = data.iloc[101:150,1:701].values

y_train = data.iloc[0:100,701].values
y_test = data.iloc[101:150,701].values

In [0]:
y_train = y_train.ravel()
y_test = y_test.ravel()

In [0]:
from sklearn.preprocessing import StandardScaler

sc_x = StandardScaler()
sc_y = StandardScaler()
x_tr = sc_x.fit_transform(x_train)
x_te = sc_x.fit_transform(x_test)
y_tr = sc_y.fit_transform(y_train.reshape(-1,1)).ravel()
y_te = sc_y.fit_transform(y_test.reshape(-1,1)).ravel()

In [8]:
x_tr.shape

(100, 700)

In [9]:
y_tr.shape

(100,)

In [0]:
"""Using Libraly"""

import numpy as np
import random
from itertools import combinations as cb
import math
from copy import deepcopy as dc
from tqdm import tqdm
from sklearn import svm  
from sklearn import model_selection as ms


In [0]:
"""Evaluate Function """
class Evaluate:
    def __init__(self):
        None
    def evaluate(self,gen):
        None
    def check_dimentions(self,dim):
        None

In [0]:
"""Common Function"""
def random_search(n,dim):
    """
    create genes list
    input:{ n: Number of population, default=20
            dim: Number of dimension
    }
    output:{genes_list → [[0,0,0,1,1,0,1,...]...n]
    }
    """
    gens=[[0 for g in range(dim)] for _ in range(n)]
    for i,gen in enumerate(gens) :
        r=random.randint(1,dim)
        for _r in range(r):
            gen[_r]=1
        random.shuffle(gen)
    return gens


In [0]:
"""BBA"""
def BBA(Eval_Func,n=20,m_i=200,dim=None,minf=0,prog=False,qmin=0,qmax=2,loud_A=0.25,r=0.4):
    """
    input:{ Eval_Func: Evaluate_Function, type is class
            n: Number of population, default=20
            m_i: Number of max iteration, default=300
            minf: minimazation flag, default=0, 0=maximization, 1=minimazation
            dim: Number of feature, default=None
            prog: Do you want to use a progress bar?, default=False
            qmin: frequency minimum to step
            qmax: frequency maximum to step
            loud_A: value of Loudness, default=0.25
            r: Pulse rate, default=0.4, Probability to relocate near the best position
            }
    output:{Best value: type float 0.967
            Best position: type list(int) [1,0,0,1,.....]
            Nunber of 1s in best position: type int [0,1,1,0,1] → 3
            }
    """
    estimate=Eval_Func().evaluate
    if dim==None:
        dim=Eval_Func().check_dimentions(dim)
    #flag=dr
    #qmin=0
    #qmax=2
    #loud_A=0.25
    #r=0.1
    #n_iter=0
    gens_dic={tuple([0]*dim):float("-inf") if minf == 0 else float("inf")}
    q=[0 for i in range(n)]
    v=[[0 for d in range(dim)] for i in range(n)]
    #cgc=[0 for i in range(max_iter)]
    fit=[float("-inf") if minf == 0 else float("inf") for i in range(n)]
    #dr=False
    gens=random_search(n,dim)#[[random.choice([0,1]) for d in range(dim)] for i in range(n)]

    for i in range(n):
        if  tuple(gens[i]) in gens_dic:
            fit[i]=gens_dic[tuple(gens[i])]
        else:
            fit[i]=estimate(gens[i])
            gens_dic[tuple(gens[i])]=fit[i]

    if minf==0:
        maxf=max(fit)
        best_v=maxf
        best_s=gens[fit.index(max(fit))]
    elif minf==1:
        minf=min(fit)
        best_v=minf
        best_s=gens[fit.index(min(fit))]


    if prog:
        miter=tqdm(range(m_i))
    else:
        miter=range(m_i)

    for it in miter:
        #cgc[i]=maxf
        for i in range(n):
            for j in range(dim):
                q[i]=qmin+(qmin-qmax)*random.random()
                v[i][j]=v[i][j]+(gens[i][j]-best_s[j])*q[i]

                vstf=abs((2/math.pi)*math.atan((math.pi/2)*v[i][j]))

                if random.random()<vstf:
                    gens[i][j]= 0 if gens[i][j]==1 else 1
                else:
                    pass

                if random.random()>r:
                    gens[i][j]=best_s[j]

            if  tuple(gens[i]) in gens_dic:
                fnew=gens_dic[tuple(gens[i])]
            else:
                fnew=estimate(gens[i])
                gens_dic[tuple(gens[i])]=fnew

            if fnew >= fit[i] and random.random() < loud_A if minf==0 else fnew <= fit[i] and random.random() < loud_A:#max?
                gens[i]=gens[i]
                fit[i]=fnew

            if fnew>best_v if minf==0 else fnew<best_v:
                best_s=dc(gens[i])
                best_v=dc(fnew)

    return best_v,best_s,best_s.count(1)


In [0]:
class Evaluate:
    def __init__(self):
        self.train_label = y_tr
        self.train_data = x_tr
        self.test_label = y_te
        self.test_data = x_te
        self.K = 4
    def evaluate(self,gen):
        mask=np.array(gen) > 0
        al_data=np.array([al[mask] for al in self.train_data])
        al_test_data=np.array([al[mask] for al in self.test_data])
        kf = ms.KFold(n_splits=self.K)
        s = 0
        for tr_ix,te_ix in kf.split(al_data):
          clf = svm.SVR()
          clf = clf.fit(al_data[tr_ix],self.train_label[tr_ix])
          s+=clf.score(al_data[te_ix],self.train_label[te_ix])
        s/=self.K
        return s   #np.count_nonzero(self.test_l==res)/len(self.test_l)
    def check_dimentions(self,dim):
        if dim==None:
            return len(self.train_d[0])
        else:
            return dim

In [0]:
def test_score(gen,tr_x,tr_y,te_x,te_y):
    clf = svm.SVR()
    mask=np.array(gen) == 1
    al_data=np.array(x_tr[:,mask])
    al_test_data=np.array(x_te[:,mask])
    return np.mean([SVR.fit(al_data,y_tr).score(al_test_data,y_te) for i in range(4)])

In [0]:
Bs,Bg,Bl=BBA(Eval_Func=Evaluate,n=len(x_tr[0:]),m_i=200, dim = len(x_tr[0]) ,minf = 0 )

In [17]:
Bs

0.7724906199192234

In [18]:
Bl

222

In [0]:
BG = np.asarray(Bg)

In [20]:
BG.reshape(1,700)

array([[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
        1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0,
        1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
        0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
        1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
        0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
        1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [21]:
BG

array([0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

In [0]:
lst = []
for i in range(len(BG)):
  if BG[i] == 0:
    lst.append(list(data.columns)[i])

ans = data.drop(lst,axis=1)

ans.to_csv('after_bat.csv', index=False)
from google.colab import files

files.download('after_bat.csv')